In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import torch.onnx

In [2]:
#загружаем обученную модель и токенизатор
model_path = 'TaskMaster-BERT2'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

In [3]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [9]:
#пример входных данных для определения размеров
sample_task = "Deploy this model"
inputs = tokenizer(sample_task, return_tensors="pt", truncation=True, max_length=512)

In [10]:
#определение размеров входных данных
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

In [12]:
#экспортируем в формат onnx
onnx_model_path = "bert_model.onnx"
torch.onnx.export(model,
                  (input_ids, attention_mask), #входные данные
                  onnx_model_path,
                  export_params=True, #экспортировать параметры модели
                  opset_version=11,
                  do_constant_folding=True,
                  input_names=['input_ids', 'attention_mask'], #имена входных слоев
                  output_names=['outputs'], #имена выходных слоев
                  dynamic_axes={'input_ids': {0: 'batch_size', 1: 'sequence_length'}, #оси входных и выходных тензоров
                                'attention_mask': {0: 'batch_size', 1: 'sequence_length'}, #которые могут меняться
                                'outputs': {0: 'batch_size'}}
                  )

============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [31]:
#тестируем что все работает
test_task = "Купить молоко"

In [32]:
from transformers import BertTokenizer

#тот же токенизатор, что при обучении модели
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
#токенизируем текст
inputs = tokenizer(test_task, return_tensors="np", truncation=True, max_length=512)

In [33]:
import onnxruntime as ort

onnx_model_path = 'bert_model.onnx'

In [34]:
session = ort.InferenceSession(onnx_model_path)

In [35]:
input_names = [input.name for input in session.get_inputs()]
output_names = [output.name for output in session.get_outputs()]

In [36]:
import numpy as np

input_ids = inputs['input_ids'].astype(np.int64)
attention_mask = inputs['attention_mask'].astype(np.int64)

In [43]:
outputs = session.run(output_names, {
    input_names[0]: input_ids,
    input_names[1]: attention_mask
})
print(outputs)

[array([[ 0.08591038, -1.1144025 ,  1.2822582 ]], dtype=float32)]


In [44]:
probabilities = np.exp(outputs[0]) / np.sum(np.exp(outputs[0]), axis=1, keepdims=True)
print(probabilities)

[[0.21696147 0.06532709 0.7177114 ]]


In [71]:
threshold=0.1

#сравниваем два самых больших результата    
max_idx = np.argmax(probabilities)
second_max_idx = np.argsort(probabilities)[-1][-2]

#если разница меньше порога, ставим категорию другое
if probabilities[0][max_idx] - probabilities[0][second_max_idx] < threshold:
    predicted_class_id = 3
    predicted_tag = 'другое'
else:
    predicted_class_id = max_idx.item()
    if predicted_class_id == 0:
        predicted_tag = 'быт'
    elif predicted_class_id == 1:
        predicted_tag = 'работа/учеба'
    elif predicted_class_id == 2:
        predicted_tag = 'покупки'
print(predicted_tag)

покупки


In [5]:
from importlib.metadata import version
print('Flask', version('Flask'), 'numpy', version('numpy'), 'onnxruntime', version('onnxruntime'), 'transformers', version('transformers'))

Flask 3.0.3 numpy 1.26.4 onnxruntime 1.17.3 transformers 4.40.1
